In [1]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/SingleImageDataAug/BAGAN/BAGAN code"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
bagan_train_new.py  data_generating_40.ipynb  selected_index_40.npz
bagan_train.py	    __pycache__		      utils.py
balancing_gan.py    rw


In [22]:
! pip install cifar10_web

In [2]:
!cp "/content/drive/My Drive/SingleImageDataAug/BAGAN/BAGAN code/." . -a

In [3]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [4]:
from bagan_train_new import *

Using TensorFlow backend.


# Generating

# Read

In [5]:
from cifar10_web import cifar10
import numpy as np

def get_indexes(index_list, label: int = 5, drop_ratio: float = 0.4):

    drop_ratio_list = dict(zip([0.4, 0.6, 0.75, 0.9],range(4)))
    
    return index_list[label+10*drop_ratio_list[drop_ratio]]

def get_cifar_10(return_one_hot_y: bool = False):
    
    X_train, y_train, X_test, y_test = cifar10(path=None)
    
    X_train = X_train.reshape(-1,3,32,32).transpose(0,2,3,1)
    X_test = X_test.reshape(-1,3,32,32).transpose(0,2,3,1)
        
    if return_one_hot_y == False:
        y_train = np.array([np.argmax(a, axis=0) for a in y_train])
        y_test = np.array([np.argmax(a, axis=0) for a in y_test])
        
    return X_train, y_train, X_test, y_test

def get_imbalanced_dataset(X_train, y_train, label, drop_ratio):
    
    if isinstance(label,int) and isinstance(drop_ratio, float):
        label = [label]
        drop_ratio = [drop_ratio]
    else:
        label = list(label)
        drop_ratio = list(drop_ratio)
        
    assert(len(label) == len(drop_ratio))
    assert(sum([1 if i in [0.4, 0.6, 0.75, 0.9] else 0 for i in drop_ratio]) == len(drop_ratio))
    
    npzfile = np.load("selected_index_40.npz", allow_pickle = True)
    indexes = npzfile["arr_0"]
    
    if y_train.ndim == 2:
        y_train_ = np.array([np.argmax(a, axis=0) for a in y_train])
    else:
        y_train_ = y_train

    for label_, drop_ratio_, i in zip(label, drop_ratio, range(len(label))):
        
        if i == 0:
            label_index = np.where(y_train_ == label_)[0]
            sample_index = get_indexes(indexes, label = label_, drop_ratio = drop_ratio_)
            deleted_index = np.delete(label_index, sample_index)
        else:
            label_index = np.where(y_train_ == label_)[0]
            sample_index = get_indexes(indexes, label = label_, drop_ratio = drop_ratio_)
            print(deleted_index.shape)
            print(np.delete(label_index, sample_index).shape)
            deleted_index = np.concatenate((deleted_index, np.delete(label_index, sample_index)))
    
        
    X_imbalanced = np.delete(X_train, deleted_index, 0)
    y_imbalanced = np.delete(y_train, deleted_index, 0)

    X_deleted = X_train[deleted_index]
    y_deleted = y_train[deleted_index]

    return X_imbalanced, y_imbalanced, X_deleted, y_deleted

In [ ]:
X_train, y_train, X_test, y_test = get_cifar_10()
print("shape: ", X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_imbalanced ,y_train_imbalanced, X_deleted, y_deleted= get_imbalanced_dataset(X_train, y_train, label = 3, drop_ratio= 0.9)
print("shape after imbalanced: ", X_train_imbalanced.shape,y_train_imbalanced.shape,X_deleted.shape, y_deleted.shape)
print("y = 3: ", np.sum(y_train == 3))
print("y = 7: ",np.sum(y_train == 7))
print("y = 3 after imbalanced: ", np.sum(y_train_imbalanced == 3))
print("y = 7 after imbalanced: ",np.sum(y_train_imbalanced == 7))

In [6]:
X_train, y_train, X_test, y_test = get_cifar_10()
print("shape: ", X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_imbalanced ,y_train_imbalanced, X_deleted, y_deleted= get_imbalanced_dataset(X_train, y_train, label = [3,6,9], drop_ratio= [0.6,0.4,0.75])
print("shape after imbalanced: ", X_train_imbalanced.shape,y_train_imbalanced.shape,X_deleted.shape, y_deleted.shape)
print("y = 3: ", np.sum(y_train == 3))
print("y = 6: ",np.sum(y_train == 6))
print("y = 7: ",np.sum(y_train == 6))
print("y = 9: ",np.sum(y_train == 9))
print("y = 3 after imbalanced: ", np.sum(y_train_imbalanced == 3))
print("y = 6 after imbalanced: ", np.sum(y_train_imbalanced == 6))
print("y = 7 after imbalanced: ", np.sum(y_train_imbalanced == 7))
print("y = 9 after imbalanced: ",np.sum(y_train_imbalanced == 9))

shape:  (50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)
(3000,)
(2000,)
(5000,)
(3750,)
shape after imbalanced:  (41250, 32, 32, 3) (41250,) (8750, 32, 32, 3) (8750,)
y = 3:  5000
y = 6:  5000
y = 7:  5000
y = 9:  5000
y = 3 after imbalanced:  2000
y = 6 after imbalanced:  3000
y = 7 after imbalanced:  5000
y = 9 after imbalanced:  1250


In [ ]:
train_model(X_train_imbalanced ,y_train_imbalanced, X_test, y_test, unbalance = [0.6,0.4,0.75], target_classes = [3,6,9], output_dir = '/content/drive/My Drive/SingleImageDataAug/BAGAN/', epochs=1, dataset_name='CIFAR10')

Executing BAGAN.
Using dataset:  CIFAR10
read input data...
input data loaded...
Required GAN for class [3 6 9]
Class counters:  [5000, 5000, 5000, 2000, 5000, 5000, 3000, 5000, 5000, 1250]
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Curr Outshape: (None, 256, 16, 16)
gen shape: (None, 3, 32, 32)
3 32 32
(None, 32, 16, 16)
(None, 64, 16, 16)
(None, 128, 8, 8)
3 32 32
(None, 32, 16, 16)
(None, 64, 16, 16)
(None, 128, 8, 8)
uratio set to: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
dratio set to: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
gratio set to: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
BAGAN init_autoenc
BAGAN: training autoencoder
Autoencoder train epoch: 1/1

